In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [2]:
import nltk
nltk.download ("stopwords")
nltk.download ("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PAVILION\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PAVILION\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english") + list(punctuation)

In [4]:
import os

In [14]:
dirs = "Q3"
datas = []
for filename in os.listdir(dirs):
    if filename.endswith('.txt'):
        name = os.path.join(dirs, filename)
        with open("Q3/arxiv_annotate10_7_1.txt", 'r') as file:
            data = file.read()
        lines = data.splitlines()
        for line in lines:
            data = {}
            if '###' in line:
                continue
            splitted = line.split('\t')
            data['Teks'] = splitted[1]
            data['label'] = splitted[0]
            datas.append(data)

In [15]:
df = pd.DataFrame(datas)
df.head()

,Teks,label
0,The Minimum Description Length principle for o...,MISC
1,"If the underlying model class is discrete, the...",MISC
2,"For MDL, in general one can only have loss bou...",MISC
3,We show that this is even the case if the mode...,AIMX
4,We derive a new upper bound on the prediction ...,OWNX


In [31]:
df.label.value_counts()

MISC    4500
OWNX    1260
AIMX     180
CONT      90
Name: label, dtype: int64

# Dataset Splitting

In [16]:
X = df.Teks
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4824,), (1206,), (4824,), (1206,))

# Training

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from jcopml.tuning import random_search_params as rsp

In [19]:
pipeline = Pipeline([
    ('prep', TfidfVectorizer(tokenizer=word_tokenize, stop_words=sw)),
    ('algo', LogisticRegression(solver='lbfgs', n_jobs=-1, random_state=42))
])


model = RandomizedSearchCV(pipeline, rsp.logreg_params, cv=3, n_iter=50, n_jobs=-1, verbose=1, random_state=42)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.3min finished
C:\Users\PAVILION\miniconda3\envs\jcopml\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'algo__C': 0.17670169402947963, 'algo__fit_intercept': True}
1.0 1.0 1.0


In [59]:
# DOI: 10.1109/MLBDBI51377.2020.00037
Text = "Diabetes is a common disease and its early symptoms are not very noticeable, so an efficient method of prediction will help patients make a self-diagnosis. However, the conventional method to identify diabetes is to make a blood glucose test by doctors and the medical resource is limited. Therefore, most patients cannot get the diagnosis immediately. Since the early symptoms of diabetes are not obvious and the relationship between symptoms and diabetes is complex, the self-diagnosis results based on patients' own experience are not accurate. The process of Machine Learning is to train a computational algorithm for prediction based on a big dataset. It is popular for its efficiency and accuracy. Also, it has the advantage of dealing with tons of data, so we can make diagnoses for plenty of patients in a short time and the result will be more accurate. In this study, we used six classical machine learning models, including logistic regression, support vector machine, decision tree, random forest, boosting and neural network, to make a prediction model for diabetes diagnosis. Our data was from UCI Machine Learning Repository, which was collected by direct questionnaires from the patients of the Sylhet Diabetes Hospital in Sylhet, Bangladesh and approved by a doctor. We conduct parameter tuning on each model to tradeoff between the accuracy and complexity. The testing error shows that random forest, boosting and neural network had better performances than logistic regression, support vector machine and decision tree. The accuracy of neural network of the test dataset achieves 96 percent, which is the best model among these models for predicting diabetes."
# DOI: 10.1109/CVIDL51233.2020.00-85
# Text = "With the rapid development of Chinnes highway transportation industry, the problem of road traffic safety has become increasingly prominent. Highway passenger transport accidents are generally fatal and fatal accidents. Traffic accidents not only cause enormous economic losses to transport enterprises, but also have a very bad social impact on local highway transport management departments, which has even become a new social instability factor. This is mainly because there are many problems in autopilot technology, such as low recognition accuracy, poor real-time performance, weak anti-interference ability and so on. However, embedded technology and machine learning can solve these problems well, so autopilot technology will become the mainstream in the future. Firstly, this paper analyses the importance of autopilot technology. Then this paper analyses the machine learning target recognition, vehicle automatic driving system model and vehicle automatic driving system flow. Finally, this paper designs the function of autopilot system."
lines = Text.split('.')

In [60]:
for line in lines:
    print(line)
    print(model.predict([line]), model.predict_proba([line]))
    print('=======================================================')

Diabetes is a common disease and its early symptoms are not very noticeable, so an efficient method of prediction will help patients make a self-diagnosis
['MISC'] [[0.0118335  0.00697561 0.84922628 0.13196462]]
 However, the conventional method to identify diabetes is to make a blood glucose test by doctors and the medical resource is limited
['MISC'] [[0.02601377 0.06274036 0.72240344 0.18884242]]
 Therefore, most patients cannot get the diagnosis immediately
['MISC'] [[0.01093048 0.00628503 0.90165988 0.08112461]]
 Since the early symptoms of diabetes are not obvious and the relationship between symptoms and diabetes is complex, the self-diagnosis results based on patients' own experience are not accurate
['OWNX'] [[0.01700236 0.01009333 0.47503416 0.49787015]]
 The process of Machine Learning is to train a computational algorithm for prediction based on a big dataset
['MISC'] [[0.00926688 0.0057448  0.87697516 0.10801316]]
 It is popular for its efficiency and accuracy
['MISC'] [[0